In [1]:
import torch
import torchvision
from torchvision import datasets
from torchvision.transforms import transforms
from torchsummary import summary
import tltorch

import numpy as np

/home/jetzeschuurman/gitProjects/phd/tltorch/tltorch/factorized_tensors/core.py:145: UserWarning: Creating a subclass of FactorizedTensor TensorizedTensor with no name.
  warnings.warn(f'Creating a subclass of FactorizedTensor {cls.__name__} with no name.')


In [2]:
path = "/bigdata/cifar10/logs/baselines/1646668631/rn18_18_dNone_128_adam_l0.001_g0.1_w0.0_sTrue/cnn_best.pth"
model = torch.load(path)

In [3]:
# model = model.cpu()

In [4]:
model.layer4[0].conv2

Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [5]:
conv = model.layer4[0].conv2
rank = 0.5
decompose_weights = True
factorization = 'cp'

conv_cp = tltorch.FactorizedConv.from_conv(
    conv, 
    rank=rank, 
    decompose_weights=decompose_weights, 
    factorization=factorization,
    decomposition_kwargs={"init":"random"},
)

In [11]:
shape = conv_cp.weight.shape

rank = conv_cp.rank

sum(shape*rank)/np.prod(shape)


0.49987369113498265

In [12]:
conv_tucker = tltorch.FactorizedConv.from_conv(
    conv, 
    rank=rank, 
    decompose_weights=decompose_weights, 
    factorization='tucker',
    # decomposition_kwargs={"init":"random"},
)

/home/jetzeschuurman/gitProjects/phd/tddl/venv/lib/python3.8/site-packages/tensorly/tucker_tensor.py:357: RuntimeWarning: Given only one int for 'rank' for decomposition a tensor of order 4. Using this rank for all modes.
  warnings.warn(message, RuntimeWarning)


RuntimeError: [enforce fail at CPUAllocator.cpp:67] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 6875146202500 bytes. Error code 12 (Cannot allocate memory)

In [8]:
conv_tucker.rank

(388, 388, 2, 2)

In [9]:
conv_tucker_1 = tltorch.FactorizedConv.from_conv(
    conv, 
    rank=1.0, 
    decompose_weights=decompose_weights, 
    factorization='tucker',
    # decomposition_kwargs={"init":"random"},
)
conv_tucker_1.rank

(483, 483, 3, 3)

In [21]:
conv_tucker_spatial = tltorch.FactorizedConv.from_conv(
    conv, 
    rank=0.5, 
    decompose_weights=decompose_weights, 
    factorization='tucker',
    # decomposition_kwargs={"init":"random"},
    fixed_rank_modes='spatial',
)

/home/jetzeschuurman/gitProjects/phd/tddl/venv/lib/python3.8/site-packages/tensorly/backend/core.py:885: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


In [22]:
rank = conv_tucker_spatial.rank
print(rank)
shape = conv_tucker_spatial.weight.shape
print(shape)

(shape[0]*rank[0]+shape[1]*rank[1]+np.prod(rank))/np.prod(shape)

(310, 310, 3, 3)
(512, 512, 3, 3)


0.5011410183376737

In [23]:
conv_tucker_spatial_custom = tltorch.FactorizedConv.from_conv(
    conv, 
    rank=(310, 310, 3, 3), 
    decompose_weights=decompose_weights, 
    factorization='tucker',
    # decomposition_kwargs={"init":"random"},
    fixed_rank_modes='spatial',
)

/home/jetzeschuurman/gitProjects/phd/tddl/venv/lib/python3.8/site-packages/tensorly/backend/core.py:885: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


In [25]:
conv_tucker_same = tltorch.FactorizedConv.from_conv(
    conv, 
    rank='same', 
    decompose_weights=decompose_weights, 
    factorization='tucker',
    # decomposition_kwargs={"init":"random"},
)
conv_tucker_same.rank

/home/jetzeschuurman/gitProjects/phd/tddl/venv/lib/python3.8/site-packages/tensorly/backend/core.py:885: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


(483, 483, 3, 3)

In [5]:
"".split(sep=',')

['']

In [31]:
conv_tucker_same_custom = tltorch.FactorizedConv.from_conv(
    conv, 
    rank=np.array(conv.weight.shape), 
    decompose_weights=decompose_weights, 
    factorization='tucker',
    # decomposition_kwargs={"init":"random"},
)
conv_tucker_same_custom.rank

/home/jetzeschuurman/gitProjects/phd/tddl/venv/lib/python3.8/site-packages/tensorly/backend/core.py:885: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


(512, 512, 3, 3)

In [12]:
conv_tucker_spatial.shape

(512, 512, 3, 3)